<a href="https://colab.research.google.com/github/cowsilver57/sessac_test/blob/main/lending_cluv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
data = pd.read_csv('/content/lending_club.csv')
data.head()

,loan_amnt,term,int_rate,installment,sub_grade,emp_length,home_ownership,annual_inc,verification_status,issue_d,...,dti,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,application_type,mort_acc,pub_rec_bankruptcies
0,10000,36 months,11.44,329.48,B4,10+ years,RENT,117000.0,Not Verified,Jan-15,...,26.24,16,0,36369,41.8,25,w,INDIVIDUAL,0.0,0.0
1,8000,36 months,11.99,265.68,B5,4 years,MORTGAGE,65000.0,Not Verified,Jan-15,...,22.05,17,0,20131,53.3,27,f,INDIVIDUAL,3.0,0.0
2,15600,36 months,10.49,506.97,B3,< 1 year,RENT,43057.0,Source Verified,Jan-15,...,12.79,13,0,11987,92.2,26,f,INDIVIDUAL,0.0,0.0
3,7200,36 months,6.49,220.65,A2,6 years,RENT,54000.0,Not Verified,Nov-14,...,2.60,6,0,5472,21.5,13,f,INDIVIDUAL,0.0,0.0
4,24375,60 months,17.27,609.33,C5,9 years,MORTGAGE,55000.0,Verified,Apr-13,...,33.95,13,0,24584,69.8,43,f,INDIVIDUAL,1.0,0.0


#1. 데이터 분류

In [ ]:
COL_DEL = ['grade','emp_title','title','earliest_cr_line','address']
COL_NUM = ['loan_amnt','term','int_rate','installment','annual_inc','dti','open_acc','pub_rec','revol_bal','revol_util','total_acc','mort_acc','pub_rec_bank']
COL_CAT = ['sub_grade','emp_length','home_ownership','verification_status','issue_d','loan_status','purpose','initial_list_status','application_type']
COL_Y = []

#2. 종속변수 인코딩

In [ ]:
loan_status_mapping = {'Fully Paid':1, 'Charged Off':0}
data['loan_status'] = data['loan_status'].map(loan_status_mapping)

In [ ]:
ratio_data = data['loan_status'].value_counts(normalize=True).reset_index()
ratio_data.columns = ['loan_status', 'ratio']

# 결과 확인
print(ratio_data)

   loan_status     ratio
0            1  0.803871
1            0  0.196129


In [ ]:
data['loan_status'].value_counts()

total_count = len(data['loan_status'])
count_0 = data['loan_status'].value_counts()[0]
count_1 = data['loan_status'].value_counts()[1]

ratio_0 = count_0 / total_count
ratio_1 = count_1 / total_count

print("Ratio of 0:", ratio_0)
print("Ratio of 1:", ratio_1)

Ratio of 0: 0.1961290811302174
Ratio of 1: 0.8038709188697826


# 3. 수치형 변수 결측치 제거 (revol_util, pub_rec_bankruptcies, mort_acc)

In [ ]:
#subset을 설정하지 않으면 전체 data의 모든 결측치를 제거함
data.dropna(subset=['revol_util'], inplace=True)

In [ ]:
#subset을 설정하지 않으면 전체 data의 모든 결측치를 제거함
data.dropna(subset=['pub_rec_bankruptcies'], inplace=True)

In [ ]:
total_acc_avg = data.groupby(by='total_acc').mean().mort_acc

def fill_mort_acc(total_acc, mort_acc):
    if np.isnan(mort_acc):
        return total_acc_avg[total_acc].round()
    else:
        return mort_acc

data['mort_acc'] = data.apply(lambda x: fill_mort_acc(x['total_acc'], x['mort_acc']), axis=1)

<ipython-input-8-2e88c7e39b00>:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  total_acc_avg = data.groupby(by='total_acc').mean().mort_acc


수치평 변수 결측치 제거 후 최종 결과

In [ ]:
data.isnull().sum()

loan_amnt                   0
term                        0
int_rate                    0
installment                 0
sub_grade                   0
emp_length              18290
home_ownership              0
annual_inc                  0
verification_status         0
issue_d                     0
loan_status                 0
purpose                     0
dti                         0
open_acc                    0
pub_rec                     0
revol_bal                   0
revol_util                  0
total_acc                   0
initial_list_status         0
application_type            0
mort_acc                    0
pub_rec_bankruptcies        0
dtype: int64

#4. 명목형 변수 인코딩

##4-1. grade

In [ ]:
grades = ['A1', 'A2', 'A3', 'A4', 'A5', 'B1', 'B2', 'B3', 'B4', 'B5', 'C1', 'C2', 'C3', 'C4', 'C5', 'D1', 'D2', 'D3', 'D4', 'D5',
          'E1', 'E2', 'E3', 'E4', 'E5', 'F1', 'F2', 'F3', 'F4', 'F5', 'G1', 'G2', 'G3', 'G4', 'G5']

# 각 등급에 부여할 값 생성
values = [34, 33, 32, 31, 30, 29, 28, 27, 26, 25, 24, 23, 22, 21, 20, 19, 18, 17,
          16, 15, 14, 13, 12, 11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 0]

# 등급과 값의 매핑을 딕셔너리로 생성
grade_to_value = dict(zip(grades, values))

# '등급' 컬럼을 매핑하여 새로운 '값' 컬럼 생성
data['sub_grade'] = data['sub_grade'].map(grade_to_value)

# 결과 출력
print(data)

        loan_amnt        term  int_rate  installment  sub_grade emp_length  \
0           10000   36 months     11.44       329.48         26  10+ years   
1            8000   36 months     11.99       265.68         25    4 years   
2           15600   36 months     10.49       506.97         27   < 1 year   
3            7200   36 months      6.49       220.65         33    6 years   
4           24375   60 months     17.27       609.33         20    9 years   
...           ...         ...       ...          ...        ...        ...   
396025      10000   60 months     10.99       217.38         26    2 years   
396026      21000   36 months     12.29       700.42         24    5 years   
396027       5000   36 months      9.99       161.32         29  10+ years   
396028      21000   60 months     15.31       503.02         23  10+ years   
396029       2000   36 months     13.61        67.98         23  10+ years   

       home_ownership  annual_inc verification_status issue_d  

##4-2. emp_length

In [ ]:
data['emp_length'].unique()

array(['10+ years', '4 years', '< 1 year', '6 years', '9 years',
       '2 years', '3 years', '8 years', '7 years', '5 years', '1 year',
       nan], dtype=object)

In [ ]:
data['emp_length'].value_counts()

10+ years    125876
2 years       35741
3 years       31598
< 1 year      31564
5 years       26455
1 year        25785
4 years       23903
6 years       20803
7 years       20783
8 years       19131
9 years       15290
Name: emp_length, dtype: int64

In [ ]:
for year in data.emp_length.unique():
    print(f"{year} years in this position:")
    print(f"{data[data.emp_length == year].loan_status.value_counts(normalize=True)}")
    print('==========================================')

10+ years years in this position:
1    0.815827
0    0.184173
Name: loan_status, dtype: float64
4 years years in this position:
1    0.807848
0    0.192152
Name: loan_status, dtype: float64
< 1 year years in this position:
1    0.793055
0    0.206945
Name: loan_status, dtype: float64
6 years years in this position:
1    0.810652
0    0.189348
Name: loan_status, dtype: float64
9 years years in this position:
1    0.799477
0    0.200523
Name: loan_status, dtype: float64
2 years years in this position:
1    0.806665
0    0.193335
Name: loan_status, dtype: float64
3 years years in this position:
1    0.804545
0    0.195455
Name: loan_status, dtype: float64
8 years years in this position:
1    0.800324
0    0.199676
Name: loan_status, dtype: float64
7 years years in this position:
1    0.805177
0    0.194823
Name: loan_status, dtype: float64
5 years years in this position:
1    0.807749
0    0.192251
Name: loan_status, dtype: float64
1 year years in this position:
1    0.800853
0    0.19914

In [ ]:
data.drop('emp_length', axis=1, inplace=True)

##4-3. home_ownership

In [ ]:
from sklearn.preprocessing import LabelEncoder

data['home_ownership'].unique()

label_mapping = {'OWN':1, 'RENT':0, 'MORTGAGE':0, 'OTHER':0, 'ANY':0, 'NONE':0}
data['home_ownership'] = data['home_ownership'].map(label_mapping)
print(data['home_ownership'])

0         0
1         0
2         0
3         0
4         0
         ..
396025    0
396026    0
396027    0
396028    0
396029    0
Name: home_ownership, Length: 395219, dtype: int64


##4-4. verification_status

In [ ]:
data['verification_status'].unique()

array(['Not Verified', 'Source Verified', 'Verified'], dtype=object)

In [ ]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
encoder.fit(data['verification_status'])
data['verification_status'] = encoder.transform(data['verification_status'])

##4-5. issue_d

In [ ]:
data.drop('issue_d', axis=1, inplace=True)

##4-6. purpose

In [ ]:
import pandas as pd
df = pd.DataFrame(data)

# 'purpose'와 'loan_status'에 대한 피벗 테이블 생성
pivot_table = df.pivot_table(index='purpose', columns='loan_status', aggfunc='size', fill_value=0).reset_index()
print(pivot_table)
# 결과 확인
pivot_table['ratio'] = pivot_table[1] / (pivot_table[0] + pivot_table[1])
pivot_table

loan_status             purpose      0       1
0                           car    628    4042
1                   credit_card  13864   69059
2            debt_consolidation  48573  185596
3                   educational     40     206
4              home_improvement   4076   19885
5                         house    434    1763
6                major_purchase   1442    7314
7                       medical    907    3268
8                        moving    667    2175
9                         other   4471   16588
10             renewable_energy     77     252
11               small_business   1664    3992
12                     vacation    463    1979
13                      wedding    217    1577


loan_status,purpose,0,1,ratio
0,car,628,4042,0.865525
1,credit_card,13864,69059,0.832809
2,debt_consolidation,48573,185596,0.792573
3,educational,40,206,0.837398
4,home_improvement,4076,19885,0.829890
5,house,434,1763,0.802458
6,major_purchase,1442,7314,0.835313
7,medical,907,3268,0.782754
8,moving,667,2175,0.765306
9,other,4471,16588,0.787692


In [ ]:
data.drop('purpose', axis=1, inplace=True)

##4-7. initial_list_status

In [ ]:
df = pd.DataFrame(data)

# 'initial_list_status'와 'loan_status'에 대한 피벗 테이블 생성
pivot_table1 = df.pivot_table(index='initial_list_status', columns='loan_status', aggfunc='size', fill_value=0)

# 결과 확인
pivot_table1['ratio'] = pivot_table1[1] / (pivot_table1[0] + pivot_table1[1])
pivot_table1


loan_status,0,1,ratio
initial_list_status,,,
f,45827,191519,0.806919
w,31696,126177,0.799231


In [ ]:
data.drop('initial_list_status', axis=1, inplace=True)

##4-8. application_type

In [ ]:
df = pd.DataFrame(data)

# 'purpose'와 'loan_status'에 대한 피벗 테이블 생성
pivot_table = df.pivot_table(index='application_type', columns='loan_status', aggfunc='size', fill_value=0).reset_index()

# 결과 확인
pivot_table['ratio'] = pivot_table[1] / (pivot_table[0] + pivot_table[1])
pivot_table

loan_status,application_type,0,1,ratio
0,DIRECT_PAY,102,184,0.643357
1,INDIVIDUAL,77367,317141,0.803890
2,JOINT,54,371,0.872941
